# Item 46: Use Descriptors for Reusable `@property` Methods

Methods in a class that are decorated by `@property` are not reusable in multiple attri of the same class. The also can't be reused by unrelated classes.

In [2]:
class Homework:
    def __init__(self):
        self._grade = 0

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value

galileo = Homework()
galileo.grade = 95

In [4]:
class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0

    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
    
    # This quickly gets tedious. For each section of the exam I need to add a new @property and related
    # validation
    @property
    def writing_grade(self):
        return self._writing_grade

    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade -= value

    @property
    def math_grade(self):
        return self._math_grade

    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value


This approach is not general. If we want to use this percentage validation in other classes beyond homework and exams, we'll need to write the `@property` boilerplate and `_check_grade` method over and over again. 

The better way to do this in Python is by using a *descriptor*. The *descriptor protocol* defines how attribute access is interpreted by the language. A descriptor class can provide `__get__` and `__set__` methods that let us reuse the grade validation behavior without the boilerplate. For this purpose, descriptors are also better than mix-ins, because they let yo reuse the same logic for many different attributes in a single class.

In [5]:
# Here, we define a new class called Exam with class attributes that are Grade instances. The Grade
# class implements the descriptor protocol
class Grade:
    def __get__(self, instance, instance_type):
        pass

    def __set__(self, instance, value):
        pass

class Exam:
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [6]:
# Here's what Python does when such descriptor attributes are accessed on an Exam instance.
# When we assign a property
exam = Exam()
exam.writing_grade = 40

# it is interpreted by Python as
Exam.__dict__['writing_grade'].__set__(exam, 40)

# When we retrive a property
exam.writing_grade

# it is interpreted as 
Exam.__dict__['writing_grade'].__get__(exam, Exam)

What drives this behavior is the `__getattribute__` method o `object`. In short, when an `Exam` instance doesn't have an attribute named `writing_grade`, Python falls back to the `Exam` class's attribute instead. If this is an object that has `__get__` and `__set__` methods, Python assumes that we want to follow the descriptor protocol.

In [7]:
# First attempt at implementing the Grade descriptor
class Grade:
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, intance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value

In [8]:
# Unfortunately, this is wrong and results in broken behavior. Accessing multiple attributes on a single
# Exam instance works as expected
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


In [10]:
# But accessing these attributes on multiple Exam instances causes unexpcted behavior
second_exam = Exam()
second_exam.writing_grade = 75
print(f'Second {second_exam.writing_grade} is right')
print(f'First {first_exam.writing_grade} is wrong; should be 82')

Second 75 is right
First 75 is wrong; should be 82


Problem: single `Grade` instance is shared across all `Exam` instances for the class attribute `writing_grade`. The `Grade` instance for this attribute is constructed once in the program lifetime, when the `Exam` class is first defined, not each time an `Exam` instance is created.

Solution: `Grade` class needs to keep track of its value for each unique `Exam` instance. 

In [11]:
# We can implement the solution above by saving the per-instance state in a dictionary
class Grade:
    def __init__(self):
        self._values = {}

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

The implementation above is simple and worls, but it still has one gotcha: It leaks memory.

The solution is to use Python's `weakref` built-in module. This module provides a special class called `WeakKeyDictionary` that can take the place of the simple dictionary use for `_values`. 

In [16]:
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

In [18]:
# Using the above implementation, everything works as expected
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()


first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print(f'First {first_exam.writing_grade} is right')
print(f'Second {second_exam.writing_grade} is right')


First 82 is right
Second 75 is right
